In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.layers import Conv2D, BatchNormalization,Flatten, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model, load_model

In [2]:
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLOR_MAP

In [3]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [4]:
def conv_block(input, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(input)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x

In [6]:
def encoder_block(input, num_filters):
  x = conv_block(input, num_filters)
  p = MaxPool2D((2, 2))(x)

  return x, p

In [7]:
def decoder_block(input, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

In [8]:
def build_unet(input_shape, num_classes):
  inputs = Input(input_shape)

  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  output = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

  model = Model(inputs, output, name="U_Net")

  return model

In [57]:
if __name__ == "__main__":
  input_shape = (512, 512, 3)
  model = build_unet(input_shape, 20)
  model.summary()

Model: "U_Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 512, 512, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_10 (Ba  (None, 512, 512, 64)         256       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_10 (Activation)  (None, 512, 512, 64)         0         ['batch_normalization_10[0

In [9]:
def load_dataset(data_path, split=0.2):
  images = sorted(glob(os.path.join(data_path, "Training", "Images", "*")))[0:10000]
  masks = sorted(glob(os.path.join(data_path, "Training", "Categories", "*")))[0:10000]

  #print(f"Images : {len(images)}, Masks :{len(masks)}")
  #print(images[0])
  #print(masks[0])

  split_size = int(len(images) * split)
    
  X_train, X_val = train_test_split(images, test_size=split_size, random_state=42)
  y_train, y_val = train_test_split(masks, test_size=split_size, random_state=42)

  X_train, X_test = train_test_split(X_train, test_size=split_size, random_state=42)
  y_train, y_test = train_test_split(y_train, test_size=split_size, random_state=42)

  return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [10]:
def get_colormap(path):
  mat_path = os.path.join(path, "human_colormap.mat")
  color_map = scipy.io.loadmat(mat_path)["colormap"]
  #print(color_map)
  #print(len(color_map))
  color_map = color_map * 256
  color_map = color_map.astype(np.uint8)
  color_map = [[c[2], c[1], c[0]] for c in color_map]
  #print(color_map)
  classes = [
      "Backgraoun",
      "Hat",
      "Hair",
      "Glove",
      "Sunglasses",
      "Dress",
      "Coat",
      "Socks",
      "Pants",
      "Torso-skon",
      "Scarf",
      "Shirt",
      "Faca",
      "Left-arm",
      "Right-arm",
      "Left-show",
      "Right-show"
  ]

  return classes, color_map

In [11]:
def read_images(path):
  #path = tf.io.read_file(path)
  #path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = cv2.resize(x , (IMG_W, IMG_H))
  x = x / 255.0
  x = x.astype(np.float32)
  return x

In [12]:
def read_mask(path):
  #path = tf.io.read_file(path)
  #path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = cv2.resize(x , (IMG_W, IMG_H))

  outputs = []
  for i, color in enumerate(COLOR_MAP):
    cmap = np.all(np.equal(x, color), axis=-1)
    #print(cmap)
    cv2.imwrite(f"cmap_{i}.png", cmap * 255)
    outputs.append(cmap)

  outputs = np.stack(outputs, axis=-1)
  outputs = outputs.astype(np.uint8)

  return outputs

In [13]:
def tf_parse(x, y):
  def _parse(x, y):
    x = x.decode()
    y = y.decode()
    x = read_images(x)
    y = read_mask(y)
    return x, y

  image, mask = tf.numpy_function(_parse, [x, y], [tf.float32, tf.uint8])
  image.set_shape([IMG_H, IMG_W, 3])
  mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])
  return image, mask

In [26]:
def tf_dataset(X, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [113]:
path = "/content/drive/MyDrive/Data"

dataset_path = r"E:\python\segmentation\Computer Vision\UNET\data\instance-level-human-parsing\instance-level_human_parsing\instance-level_human_parsing"

if __name__ == "__main__":
  np.random.seed(42)
  tf.random.set_seed(42)

  create_dir(dataset_path + "/files")

  IMG_H = 320
  IMG_W = 416
  NUM_CLASSES = 20

  input_shape = (IMG_H, IMG_W, 3)

  batch_size = 2
  learning_rate = 1e-4
  num_epochs = 1

  #dataset_path = "/content/drive/MyDrive/Data/instance-level-human-parsing/instance-level_human_parsing/instance-level_human_parsing"

  model_path = os.path.join(dataset_path, "/files", "Segmentation_100epochs_model.h5")

  csv_path = os.path.join(dataset_path, "/files", "Segmentation_csv_log.csv")

  (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_dataset(dataset_path)

  #print(f"Train: {len(X_train)} / {len(y_train)} - Valid : {len(X_val)} / {len(y_val)} - Test : {len(X_test)} / {len(y_test)}")

  CLASSES, COLOR_MAP = get_colormap(dataset_path)

  train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
  val_dataset = tf_dataset(X_val, y_val, batch=batch_size)

  model = build_unet(input_shape, NUM_CLASSES)
  model.compile(
      loss="categorical_crossentropy",
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
  )
  #model.summary()

  callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path, append=True),
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
    ]

  model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=val_dataset,
        callbacks=callbacks
    )

3000/3000 [==============================] - ETA: 0s - loss: 1.2802
Epoch 1: val_loss improved from inf to 1.11949, saving model to E:/files\Segmentation_100epochs_model.h5


C:\Users\hamid\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3000/3000 [==============================] - 14792s 5s/step - loss: 1.2802 - val_loss: 1.1195 - lr: 1.0000e-04


In [119]:
model.save("Instance_level_human_Segmentation.h5")

In [28]:
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score

In [30]:
def gratscale_to_rgb(y_pred, CLASSES, colormap):
    h, w, _ = y_pred.shape
    y_pred = y_pred.astype(np.int32)

    output = []
    for i, pixel in enumerate(y_pred.flatten()):
        output.append(colormap[pixel])

    output = np.reshape(output, (h, w, 3))
    return output

In [32]:
def save_results(image, mask, y_pred, save_image_path):
  h, w, _ = image.shape
  line = np.ones((h, 10, 3)) * 255

  y_pred = np.expand_dims(y_pred, axis=-1)
  y_pred = gratscale_to_rgb(y_pred, CLASSES, COLOR_MAP)

  cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
  cv2.imwrite(save_image_path, cat_images)

In [38]:
dataset_path = r"E:\python\segmentation\Computer Vision\UNET\data\instance-level-human-parsing\instance-level_human_parsing\instance-level_human_parsing"


In [46]:
data_path = r"E:\python\segmentation\Computer Vision\UNET\data\instance-level-human-parsing\instance-level_human_parsing\instance-level_human_parsing\files"
saved_model = r"E:\python\segmentation\Computer Vision\UNET\data"
if __name__=="__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(data_path + "/results")

    IMG_H = 320
    IMG_W = 416
    NUM_CLASSES = 20

    #dataset_path = "/content/drive/MyDrive/Data/instance-level-human-parsing/instance-level_human_parsing"

    model_path= os.path.join(saved_model, "Instance_level_human_Segmentation.h5")
    model = load_model(model_path, compile=False)

    CLASSES, COLOR_MAP = get_colormap(dataset_path)

    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_dataset(dataset_path)


    #print(f"Train: {len(X_train)} / {len(y_train)} - Valid : {len(X_val)} / {len(y_val)} - Test : {len(X_test)} / {len(y_test)}")

    X_test = X_test[:10]
    y_test = y_test[:10]

    score = []
    for x, y in tqdm(zip(X_test, y_test), total=len(X_test)):
        name = x.split("\\")[-1].split(".")[0]
        print(name)

        image = cv2.imread(x, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IMG_W, IMG_H))
        image_x = image
        image = image / 255.0
        #x = x.astype(np.float32)
        image = np.expand_dims(image, axis=0)

        mask = cv2.imread(y, cv2.IMREAD_COLOR)
        mask = cv2.resize(mask, (IMG_W, IMG_H))

        mask_x = mask

        onehot_mask = []

        for color in COLOR_MAP:
            cmap = np.all(np.equal(mask, color), axis=-1)
            onehot_mask.append(cmap)

        onehot_mask = np.stack(onehot_mask, axis=-1)
        onehot_mask = np.argmax(onehot_mask, axis=-1)
        onehot_mask = onehot_mask.astype(np.int32)

        my_path = r"E:\python\segmentation\Computer Vision\UNET\data"      

        cv2.imwrite(my_path + "\\rgn.png", mask_x)
        #cv2.imwrite(my_path + "\\rgn_mask.png", onehot_mask * (255/20))

        pred = model.predict(image, verbose=0)[0]
        #print(pred)
        pred = np.argmax(pred, axis=-1)
        #print(pred)
        pred = pred.astype(np.float32)

        #cv2.imwrite(my_path + "\\pred.png", pred * (255/20))

        save_image_path = os.path.join(my_path, f"{name}.png")
        save_results(image_x, mask, pred, save_image_path)

        onehot_mask = onehot_mask.flatten()
        pred = pred.flatten()

        labels = [i for i in range(NUM_CLASSES)]

        f1_value = f1_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)
        jac_value = jaccard_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)

        #print(f1_value)
        score.append([f1_value, jac_value])

    score = np.array(score)
    score = np.mean(score, axis=0)
    print(score.shape)

    f = open(data_path + "\\score.csv", "w")
    f.write("Class", "F1", "Jaccard\n")

    l = ["Class", "F1", "Jaccard"]
    print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s}")
    print("_"*35)

    for i in range(score.shape[1]):
        class_name = CLASSES[1]
        f1 = score[0, i]
        jac = score[1, i]
        dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
        print(dstr)
        f.write(f"{class_name:15s}, {f1:1.5f}, {jac:1.5f}\n")

    print("_"*35)
    class_mean = np.mean(score, axis=-1)
    class_name = "Mean"
    f1 = class_mean[0]
    jac = class_mean[1]
    dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    print(dstr)
    f.write(f"{class_name:15s}, {f1:1.5f}, {jac:1.5f}\n")

    f.close()

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

0004097


 10%|████████▎                                                                          | 1/10 [00:04<00:37,  4.15s/it]

0010674


 20%|████████████████▌                                                                  | 2/10 [00:05<00:17,  2.25s/it]

...

0001518


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:11<00:00,  1.01it/s]

0004331


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.25s/it]

(2, 20)
